In [13]:
base_path = r'C:\Users\dhutt\Desktop\SCHEDULING_MODEL\code\reworked_data_model\results\comparison\\'
benchmark_base_path = r'C:\Users\dhutt\Desktop\SCHEDULING_MODEL\code\external_test_data\FJSSPinstances\\'
files = ['s0_s1_3600.txt','s4_3600.txt','s5_s6_s3_3600.txt']

In [14]:
def read_benchmark_file(source, instance):
    if source.startswith('0'):
        target_file = f'Behnke{instance}.fjs'
    elif source.startswith('1'):
        target_file = f'BrandimarteMk{instance}.fjs'
    elif source.startswith('2a'):
        target_file = f'HurinkSdata{instance}.fjs'
    elif source.startswith('2b'):
        target_file = f'HurinkEdata{instance}.fjs'
    elif source.startswith('2c'):
        target_file = f'HurinkRdata{instance}.fjs'
    elif source.startswith('2d'):
        target_file = f'HurinkVdata{instance}.fjs'
    elif source.startswith('3'):
        target_file = f'DPpaulli{instance}.fjs'
    elif source.startswith('4'):
        target_file = f'ChambersBarnes{instance}.fjs'
    elif source.startswith('5'):
        target_file = f'Kacem{instance}.fjs'
    elif source.startswith('6'):
        target_file = f'Fattahi{instance}.fjs'
    path = benchmark_base_path + f'{source}\\{target_file}'
    return open(path, 'r').readlines()

In [15]:
def translate(content, limit):
    line = dict()
    line['source'] = content[0]
    line['instance'] = int(content[1])
    line['best_objective'] = float(content[6])
    line['generations'] = int(content[4])
    line['restarts'] = int(content[5])
    line['n_explored'] = int(float(content[3]))
    line['runtime'] = min(float(content[2]), limit)

    # add information about the benchmark instance
    benchmark_file = read_benchmark_file(line['source'], line['instance'])
    benchmark_system = benchmark_file[0].split(' ')
    line['n_machines'] = int(benchmark_system[1])
    line['n_jobs'] = int(benchmark_system[0])
    line['average_workstations_per_operation'] = float(benchmark_system[2])
    jobs = benchmark_file[1:]
    line['n_operations'] = sum([int(x.split(' ')[0]) for x in jobs])
    line['average_operations_per_job'] = line['n_operations'] / line['n_jobs']
    line['beta_flexibility'] = line['average_workstations_per_operation']/line['n_machines']
    line['approximate_max_dissimilarity'] = line['n_operations'] + line['average_operations_per_job'] * line['average_workstations_per_operation']
    line['limit'] = limit
    line['optimizer'] = 'GA'
    return line    

In [16]:
data = []
for file in files:
    content = open(base_path + file, 'r').readlines()
    for values in content:
        line = translate(values.split(';'), 3600)
        data.append(line)

In [29]:
best = dict()
for line in data:
    if line['source']+str(line['instance']) not in best:
        best[line['source']+str(line['instance'])] = line
    elif best[line['source']+str(line['instance'])]['best_objective'] > line['best_objective']:
        best[line['source']+str(line['instance'])] = line

In [30]:
print(len(best))

138


In [31]:
import pandas as pd
df = pd.DataFrame.from_dict(best.values())

In [32]:
df.loc[df['source'] == '6_Fattahi']

,source,instance,best_objective,generations,restarts,n_explored,runtime,n_machines,n_jobs,average_workstations_per_operation,n_operations,average_operations_per_job,beta_flexibility,approximate_max_dissimilarity,limit
100,6_Fattahi,1,66.0,2,0,45,0.015623,2,2,2.0,4,2.0,1.000000,8.0,3600
101,6_Fattahi,2,107.0,0,0,5,0.000000,2,2,1.5,4,2.0,0.750000,7.0,3600
102,6_Fattahi,3,221.0,63,0,1265,0.062486,2,3,1.7,6,2.0,0.850000,9.4,3600
103,6_Fattahi,4,355.0,2,0,45,0.015622,2,3,1.7,6,2.0,0.850000,9.4,3600
104,6_Fattahi,5,119.0,17,0,345,0.031241,2,3,2.0,6,2.0,1.000000,10.0,3600
105,6_Fattahi,6,320.0,5,0,105,0.031243,3,3,1.6,9,3.0,0.533333,13.8,3600
106,6_Fattahi,7,397.0,3,0,65,0.015623,5,3,2.0,9,3.0,0.400000,15.0,3600
107,6_Fattahi,8,253.0,3,0,65,0.015620,4,3,2.0,9,3.0,0.500000,15.0,3600
108,6_Fattahi,9,210.0,3,0,65,0.015621,3,3,2.0,9,3.0,0.666667,15.0,3600
109,6_Fattahi,10,516.0,3,0,65,0.015621,5,4,1.7,12,3.0,0.340000,17.1,3600


In [33]:
df.loc[df['source'] == '5_Kacem']

,source,instance,best_objective,generations,restarts,n_explored,runtime,n_machines,n_jobs,average_workstations_per_operation,n_operations,average_operations_per_job,beta_flexibility,approximate_max_dissimilarity,limit
96,5_Kacem,1,11.0,195,1,5570,0.499883,5,4,5.0,12,3.000000,1.0,27.000000,3600
97,5_Kacem,2,11.0,654,4,55215,15.246434,7,10,7.0,29,2.900000,1.0,49.300000,3600
98,5_Kacem,3,7.0,986,4,76235,20.167162,10,10,10.0,30,3.000000,1.0,60.000000,3600
99,5_Kacem,4,11.0,2906,19,3198780,3600.000000,10,15,10.0,56,3.733333,1.0,93.333333,3600


In [34]:
df.loc[df['source'] == '4_ChambersBarnes']

,source,instance,best_objective,generations,restarts,n_explored,runtime,n_machines,n_jobs,average_workstations_per_operation,n_operations,average_operations_per_job,beta_flexibility,approximate_max_dissimilarity,limit
72,4_ChambersBarnes,1,963.0,1612,10,1076605,3600.0,11,10,1.0,100,10.0,0.090909,110.0,3600
73,4_ChambersBarnes,2,948.0,1762,10,1261705,3600.0,12,10,1.0,100,10.0,0.083333,110.0,3600
74,4_ChambersBarnes,3,968.0,1815,9,1478250,3600.0,13,10,1.0,100,10.0,0.076923,110.0,3600
75,4_ChambersBarnes,4,919.0,1649,10,1132765,3600.0,12,10,1.0,100,10.0,0.083333,110.0,3600
76,4_ChambersBarnes,5,873.0,1657,10,1308325,3600.0,13,10,1.0,100,10.0,0.076923,110.0,3600
77,4_ChambersBarnes,6,951.0,1744,10,1154445,3600.0,11,10,1.0,100,10.0,0.090909,110.0,3600
78,4_ChambersBarnes,7,946.0,1489,10,1102245,3600.0,12,10,1.0,100,10.0,0.083333,110.0,3600
79,4_ChambersBarnes,8,985.0,1436,8,749755,3600.0,11,15,1.0,150,10.0,0.090909,160.0,3600
80,4_ChambersBarnes,9,977.0,1316,8,720315,3600.0,12,15,1.0,150,10.0,0.083333,160.0,3600
81,4_ChambersBarnes,10,991.0,1227,8,655195,3600.0,13,15,1.0,150,10.0,0.076923,160.0,3600


In [35]:
df.loc[df['source'] == '3_DPpaulli']

,source,instance,best_objective,generations,restarts,n_explored,runtime,n_machines,n_jobs,average_workstations_per_operation,n_operations,average_operations_per_job,beta_flexibility,approximate_max_dissimilarity,limit
120,3_DPpaulli,1,2606.0,1268,7,496240,3600.0,5,10,1.0,196,19.600000,0.200,215.600000,3600
121,3_DPpaulli,2,2364.0,1844,7,582860,3600.0,5,10,1.0,196,19.600000,0.200,215.600000,3600
122,3_DPpaulli,3,2283.0,1478,7,522000,3600.0,5,10,1.0,196,19.600000,0.200,215.600000,3600
123,3_DPpaulli,4,2600.0,1311,7,529460,3600.0,5,10,1.0,196,19.600000,0.200,215.600000,3600
124,3_DPpaulli,5,2356.0,1548,7,583200,3600.0,5,10,1.0,196,19.600000,0.200,215.600000,3600
125,3_DPpaulli,6,2272.0,1644,7,521080,3600.0,5,10,1.0,196,19.600000,0.200,215.600000,3600
126,3_DPpaulli,7,2672.0,1385,6,279985,3600.0,8,15,1.0,293,19.533333,0.125,312.533333,3600
127,3_DPpaulli,8,2197.0,1197,6,352965,3600.0,8,15,1.0,293,19.533333,0.125,312.533333,3600
128,3_DPpaulli,9,2484.0,1296,6,314725,3600.0,8,15,1.0,293,19.533333,0.125,312.533333,3600
129,3_DPpaulli,10,2585.0,1306,6,289065,3600.0,8,15,1.0,293,19.533333,0.125,312.533333,3600


In [36]:
df.loc[df['source'] == '1_Brandimarte']

,source,instance,best_objective,generations,restarts,n_explored,runtime,n_machines,n_jobs,average_workstations_per_operation,n_operations,average_operations_per_job,beta_flexibility,approximate_max_dissimilarity,limit
0,1_Brandimarte,1,40.0,269,1,8470,3.983443,6,10,2.000000,55,5.500000,0.333333,66.000000,3600
1,1_Brandimarte,2,26.0,3028,21,3518230,3600.000000,6,10,3.500000,58,5.800000,0.583333,78.300000,3600
2,1_Brandimarte,3,204.0,705,3,37880,104.975468,8,15,3.000000,150,10.000000,0.375000,180.000000,3600
3,1_Brandimarte,4,60.0,1734,12,1480295,3600.000000,8,15,2.000000,90,6.000000,0.250000,102.000000,3600
4,1_Brandimarte,5,173.0,1623,10,1314545,3600.000000,4,15,1.500000,106,7.066667,0.375000,116.600000,3600
5,1_Brandimarte,6,65.0,1685,7,995580,3600.000000,10,10,3.000000,150,15.000000,0.300000,195.000000,3600
6,1_Brandimarte,7,141.0,1773,11,1268700,3600.000000,5,20,3.000000,100,5.000000,0.600000,115.000000,3600
7,1_Brandimarte,8,523.0,1099,7,394280,3600.000000,10,20,1.500000,225,11.250000,0.150000,241.875000,3600
8,1_Brandimarte,9,324.0,1368,6,477585,3600.000000,10,20,3.000000,240,12.000000,0.300000,276.000000,3600
9,1_Brandimarte,10,251.0,1208,6,452505,3600.000000,15,20,3.000000,240,12.000000,0.200000,276.000000,3600


In [37]:
df.loc[df['source'] == '0_BehnkeGeiger']

,source,instance,best_objective,generations,restarts,n_explored,runtime,n_machines,n_jobs,average_workstations_per_operation,n_operations,average_operations_per_job,beta_flexibility,approximate_max_dissimilarity,limit
15,0_BehnkeGeiger,1,93.0,3436,23,3874440,3600.000000,20,10,6.08,50,5.0,0.304000,80.40,3600
16,0_BehnkeGeiger,2,94.0,3460,22,4096625,3600.000000,20,10,6.72,50,5.0,0.336000,83.60,3600
17,0_BehnkeGeiger,3,92.0,3584,22,4011785,3600.000000,20,10,6.24,50,5.0,0.312000,81.20,3600
18,0_BehnkeGeiger,4,99.0,3537,22,4038785,3600.000000,20,10,6.64,50,5.0,0.332000,83.20,3600
19,0_BehnkeGeiger,5,94.0,3434,23,4038500,3600.000000,20,10,6.16,50,5.0,0.308000,80.80,3600
20,0_BehnkeGeiger,6,136.0,2451,11,1931940,3600.000000,20,20,6.68,100,5.0,0.334000,133.40,3600
21,0_BehnkeGeiger,7,136.0,2275,11,1874560,3600.000000,20,20,6.52,100,5.0,0.326000,132.60,3600
22,0_BehnkeGeiger,8,133.0,2306,11,1893680,3600.000000,20,20,6.44,100,5.0,0.322000,132.20,3600
23,0_BehnkeGeiger,9,135.0,2397,11,1989420,3600.000000,20,20,6.44,100,5.0,0.322000,132.20,3600
24,0_BehnkeGeiger,10,140.0,2254,11,1885000,3600.000000,20,20,6.12,100,5.0,0.306000,130.60,3600


In [38]:
def translate_gurobi(columns, limit):
    line = dict()
    line['source'] = columns[0]
    line['instance'] = int(columns[1])
    line['best_objective'] = int(float(columns[2])+0.5)
    line['gap'] = float(columns[3])
    line['status'] = int(columns[4])
    line['n_explored'] = int(float(columns[5]))
    line['runtime'] = float(columns[6])
    line['overall_runtime'] = float(columns[7])
    line['setup_time'] = line['overall_runtime'] - line['runtime']
    # add information about the benchmark instance
    benchmark_file = read_benchmark_file(line['source'], line['instance'])
    benchmark_system = benchmark_file[0].split(' ')
    line['n_machines'] = int(benchmark_system[1])
    line['n_jobs'] = int(benchmark_system[0])
    line['average_workstations_per_operation'] = float(benchmark_system[2])
    jobs = benchmark_file[1:]
    line['n_operations'] = sum([int(x.split(' ')[0]) for x in jobs])
    line['average_operations_per_job'] = line['n_operations'] / line['n_jobs']
    line['beta_flexibility'] = line['average_workstations_per_operation']/line['n_machines']
    line['approximate_max_dissimilarity'] = line['n_operations'] + line['average_operations_per_job'] * line['average_workstations_per_operation']
    line['limit'] = limit
    line['optimizer'] = 'Gurobi'
    return line    

In [39]:
path_30_min = r'C:\Users\dhutt\Desktop\SCHEDULING_MODEL\code\reworked_data_model\results\30min_gurobi.txt'
data_30 = []
file = open(path_30_min, 'r')
file_content = file.readlines()
results_30_min = [x.split(';') for x in file_content]
for columns in results_30_min[1:]:
    line = translate(columns, 30)
    data_30.append(line)

ValueError: invalid literal for int() with base 10: '158083.0'